In [ ]:
import cv2
import os
import math
import numpy as np

def DarkChannel(im, sz):
    b, g, r = cv2.split(im)
    dc = cv2.min(cv2.min(r, g), b)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (sz, sz))
    dark = cv2.erode(dc, kernel)
    return dark

def AtmLight(im, dark):
    [h, w] = im.shape[:2]
    imsz = h * w
    numpx = int(max(math.floor(imsz / 1000), 1))
    darkvec = dark.reshape(imsz)
    imvec = im.reshape(imsz, 3)
    indices = darkvec.argsort()
    indices = indices[imsz - numpx::]

    atmsum = np.zeros([1, 3])
    for ind in range(1, numpx):
        atmsum = atmsum + imvec[indices[ind]]
    A = atmsum / numpx
    return A

def TransmissionEstimate(im, A, sz):
    omega = 0.95
    im3 = np.empty(im.shape, im.dtype)
    for ind in range(0, 3):
        im3[:, :, ind] = im[:, :, ind] / A[0, ind]
    transmission = 1 - omega * DarkChannel(im3, sz)
    return transmission

def Guidedfilter(im, p, r, eps):
    mean_I = cv2.boxFilter(im, cv2.CV_64F, (r, r))
    mean_p = cv2.boxFilter(p, cv2.CV_64F, (r, r))
    mean_Ip = cv2.boxFilter(im * p, cv2.CV_64F, (r, r))
    cov_Ip = mean_Ip - mean_I * mean_p
    mean_II = cv2.boxFilter(im * im, cv2.CV_64F, (r, r))
    var_I = mean_II - mean_I * mean_I
    a = cov_Ip / (var_I + eps)
    b = mean_p - a * mean_I
    mean_a = cv2.boxFilter(a, cv2.CV_64F, (r, r))
    mean_b = cv2.boxFilter(b, cv2.CV_64F, (r, r))
    q = mean_a * im + mean_b
    return q

def TransmissionRefine(im, et):
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    gray = np.float64(gray) / 255
    r = 60
    eps = 0.0001
    t = Guidedfilter(gray, et, r, eps)
    return t

def Recover(im, t, A, tx=0.1):
    res = np.empty(im.shape, im.dtype)
    t = cv2.max(t, tx)

    for ind in range(0, 3):
        res[:, :, ind] = (im[:, :, ind] - A[0, ind]) / t + A[0, ind]
    return res

if __name__ == '__main__':
    dataset_folder = 'Foggy Images'
    output_folder = 'output7'

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    for filename in os.listdir(dataset_folder):
        if filename.endswith('.png') or filename.endswith('.jpg'):
            input_image = os.path.join(dataset_folder, filename)
            src = cv2.imread(input_image)
            if src is None:
                print(f"Error: Unable to read the input image '{filename}'.")
                continue
            I = src.astype('float64') / 255
            dark = DarkChannel(I, 15)
            A = AtmLight(I, dark)
            te = TransmissionEstimate(I, A, 15)
            
            # Down-sample the rough transmission map and the guidance image
            te_downsampled = cv2.resize(te, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)
            src_downsampled = cv2.resize(src, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_NEAREST)
            
            cv2.imshow('Transmission Map (Before Refinement with guided filter )', te_downsampled)
            cv2.waitKey(0)
            
            t = TransmissionRefine(src_downsampled, te_downsampled)
            
            # Up-sample the refined transmission map
            t_upsampled = cv2.resize(t, (src.shape[1], src.shape[0]), interpolation=cv2.INTER_LINEAR)
            
            J = Recover(I, t_upsampled, A, 0.1)
            
            cv2.imshow('Transmission Map (After Refinement with guided filter)', t_upsampled)
            cv2.waitKey(0)
            cv2.imshow('Input Image', src)
            cv2.waitKey(0)
            cv2.imshow('Output Image', J)
            cv2.waitKey(0)
            
            # Save the output images
            output_image_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_image_path, J * 255)  # Convert back to 0-255 range before saving

    cv2.destroyAllWindows()
    print("Folder Saved")
